# Flight Delay Project Data : EDA of `WEATHER` and `STATIONS`
## Bureau of Transportation Statistics
https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236   
https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

Calendar Years: 2015 - 2019

### Additional sources
This might be useful in matching station codes to airports:
1. http://dss.ucar.edu/datasets/ds353.4/inventories/station-list.html
2. https://www.world-airport-codes.com/


https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf
https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236

In this notebook, we evaluate the `WEATHER` and `STATIONS` data only

# Imports & Helper Functions

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import databricks.koalas as ks
from pyspark.sql import SQLContext
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [0]:
############################################################################
# Plotting Utilities, Constants, Methods for W209 arXiv project
############################################################################

#---------------------------------------------------------------------------
## Plotting Palette
#
# Create a dict object containing U.C. Berkeley official school colors for plot palette 
# reference : https://brand.berkeley.edu/colors/
# secondary reference : https://alumni.berkeley.edu/brand/color-palette# CLass Initialization
#---------------------------------------------------------------------------

berkeley_palette = {'berkeley_blue'     : '#003262',
                    'california_gold'   : '#fdb515',
                    'founders_rock'     : '#3b7ea1',
                    'medalist'          : '#c4820e',
                    'bay_fog'           : '#ddd5c7',
                    'lawrence'          : '#00b0da',
                    'sather_gate'       : '#b9d3b6',
                    'pacific'           : '#46535e',
                    'soybean'           : '#859438',
                    'south_hall'        : '#6c3302',
                    'wellman_tile'      : '#D9661F',
                    'rose_garden'       : '#ee1f60',
                    'golden_gate'       : '#ed4e33',
                    'lap_lane'          : '#00a598',
                    'ion'               : '#cfdd45',
                    'stone_pine'        : '#584f29',
                    'grey'              : '#eeeeee',
                    'web_grey'          : '#888888',
                    # alum only colors
                    'metallic_gold'     : '#BC9B6A',
                    'california_purple' : '#5C3160'                   
                    }

# `Weather` Data Analysis

In [0]:
# load raw weather data into koalas dataframe and print raw info about shape and sparsity
df = ks.sql("select * from weather_tbl").to_koalas()
print(df.info())

/databricks/python/lib/python3.7/site-packages/databricks/koalas/generic.py:405: FutureWarning: `get_dtype_counts` has been deprecated and will be removed in a future version. For DataFrames use `.dtypes.value_counts()
 FutureWarning,
<class 'databricks.koalas.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 177 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 STATION 1000 non-null object 
 1 DATE 1000 non-null datetime64[ns]
 2 SOURCE 1000 non-null int16 
 3 LATITUDE 1000 non-null float64 
 4 LONGITUDE 1000 non-null float64 
 5 ELEVATION 1000 non-null float64 
 6 NAME 1000 non-null object 
 7 REPORT_TYPE 1000 non-null object 
 8 CALL_SIGN 1000 non-null object 
 9 QUALITY_CONTROL 1000 non-null object 
 10 WND 1000 non-null object 
 11 CIG 1000 non-null object 
 12 VIS 1000 non-null object 
 13 TMP 1000 non-null object 
 14 DEW 1000 non-null object 
 15 SLP 1000 non-null object 
 16 AW1 1000 non-null object 
 17 GA1 1000 non-null object 
 18 GA2 1000 non-null object 
 19 GA3 1000 non-null object 
 20 GA4 1000 non-null object 
 21 GE1 1000 non-null object 
 22 GF1 1000 non-null object 
 23 KA1 1000 non-null object 
 24 KA2 1000 non-null object 
 25 MA1 1000 non-null object 
 26 MD1 1000 non-null object 
 27 MW1 1000 non-null object 
 28 MW2 1000 non-null object 
 29 OC1 1000 non-null object 
 30 OD1 1000 non-null object 
 31 OD2 1000 non-null object 
 32 REM 1000 non-null object 
 33 EQD 1000 non-null object 
 34 AW2 1000 non-null object 
 35 AX4 1000 non-null object 
 36 GD1 1000 non-null object 
 37 AW5 1000 non-null object 
 38 GN1 1000 non-null object 
 39 AJ1 1000 non-null object 
 40 AW3 1000 non-null object 
 41 MK1 1000 non-null object 
 42 KA4 1000 non-null object 
 43 GG3 1000 non-null object 
 44 AN1 1000 non-null object 
 45 RH1 1000 non-null object 
 46 AU5 1000 non-null object 
 47 HL1 1000 non-null object 
 48 OB1 1000 non-null object 
 49 AT8 1000 non-null object 
 50 AW7 1000 non-null object 
 51 AZ1 1000 non-null object 
 52 CH1 1000 non-null object 
 53 RH3 1000 non-null object 
 54 GK1 1000 non-null object 
 55 IB1 1000 non-null object 
 56 AX1 1000 non-null object 
 57 CT1 1000 non-null object 
 58 AK1 1000 non-null object 
 59 CN2 1000 non-null object 
 60 OE1 1000 non-null object 
 61 MW5 1000 non-null object 
 62 AO1 1000 non-null object 
 63 KA3 1000 non-null object 
 64 AA3 1000 non-null object 
 65 CR1 1000 non-null object 
 66 CF2 1000 non-null object 
 67 KB2 1000 non-null object 
 68 GM1 1000 non-null object 
 69 AT5 1000 non-null object 
 70 AY2 1000 non-null object 
 71 MW6 1000 non-null object 
 72 MG1 1000 non-null object 
 73 AH6 1000 non-null object 
 74 AU2 1000 non-null object 
 75 GD2 1000 non-null object 
 76 AW4 1000 non-null object 
 77 MF1 1000 non-null object 
 78 AA1 1000 non-null object 
 79 AH2 1000 non-null object 
 80 AH3 1000 non-null object 
 81 OE3 1000 non-null object 
 82 AT6 1000 non-null object 
 83 AL2 1000 non-null object 
 84 AL3 1000 non-null object 
 85 AX5 1000 non-null object 
 86 IB2 1000 non-null object 
 87 AI3 1000 non-null object 
 88 CV3 1000 non-null object 
 89 WA1 1000 non-null object 
 90 GH1 1000 non-null object 
 91 KF1 1000 non-null object 
 92 CU2 1000 non-null object 
 93 CT3 1000 non-null object 
 94 SA1 1000 non-null object 
 95 AU1 1000 non-null object 
 96 KD2 1000 non-null object 
 97 AI5 1000 non-null object 
 98 GO1 1000 non-null object 
 99 GD3 1000 non-null object 
 100 CG3 1000 non-null object 
 101 AI1 1000 non-null object 
 102 AL1 1000 non-null object 
 103 AW6 1000 non-null object 
 104 MW4 1000 non-null object 
 105 AX6 1000 non-null object 
 106 CV1 1000 non-null object 
 107 ME1 1000 non-null object 
 108 KC2 1000 non-null object 
 109 CN1 1000 non-null object 
 110 UA1 1000 non-null object 
 111 GD5 1000 non-null object 
 112 UG2 1000 non-null object 
 113 AT3 1000 non-null object 
 114 AT4 1000 non-null object 
 115 GJ1 1000 non-null object 
 116 MV1 1000 non-null object 
 117 GA5 1000 

### Raw data profile:
 - 630,904,436 rows
 - 177 columns
   - `3 x float64`
   - `1 x int16`
   - `172 x object`

### Missing data analysis

In [0]:
# evaluate missing data
missing = df.isnull().sum(axis=0).reset_index()
missing.columns = ['column', 'total_missing']
missing['missing_pct'] = missing.total_missing / df.shape[0] * 100
missing = missing.sort_values(by=['total_missing','column']).reset_index(drop = True)
missing['category'] = ['No missing data'] * missing.shape[0]
missing.loc[((missing['missing_pct'] > 0.0) & (missing['missing_pct'] <= 40.0)), 'category'] = 'Up to 40% missing data'
missing.loc[((missing['missing_pct'] > 40.0) & (missing['missing_pct'] <= 80.0)), 'category'] = 'Up to 80% missing data'
missing.loc[missing['missing_pct'] > 80.0, 'category'] = 'Over 80% missing data'
#missing.head(5).style.set_properties(**{'background-color': berkeley_palette['california_gold'], 'color': berkeley_palette['berkeley_blue'], 'border-color': 'white'})

cats = missing.groupby('category').column.count().to_frame()
cats.columns = ['Category Count']

cats.style.bar(color = berkeley_palette['founders_rock'], align = 'mid')\
  .set_caption('Distribution of missing data per column in Weather')

Out[4]:

,Category Count
category,
No missing data,175
Up to 40% missing data,2


In [0]:
missing_subset = missing[missing.missing_pct > 0.0].copy().sort_values(by='missing_pct')
x, y, h = missing_subset['column'].to_numpy(),  missing_subset.missing_pct.to_numpy(), missing_subset.total_missing.to_numpy()
marker_colors = [berkeley_palette['pacific'] if x > 80.0 else berkeley_palette['lawrence'] for x in missing_subset.missing_pct.to_numpy()]

fig = go.Figure(data=[go.Bar(x=x, y=y, hovertext=h)])
fig.update_traces(marker_color=marker_colors, marker_line_color=berkeley_palette['berkeley_blue'],
                  marker_line_width=1.5, opacity=0.7)
fig.update_layout(title_text='[Weather]: Columns w/ Missing Values', title_font_size = 24,
                  height=450, width=800,
                  yaxis=dict(title='% Missing Values', titlefont_size=16, tickfont_size=14, color=berkeley_palette['berkeley_blue']),
                  xaxis=dict(title='Column Names', titlefont_size=16, color=berkeley_palette['berkeley_blue']),
                  plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [0]:
good_cols = ['STATION', 'DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'REPORT_TYPE', 'CALL_SIGN', 'QUALITY_CONTROL', 'WND', 'CIG', 'VIS', 
    'TMP', 'DEW', 'SLP', 'MA1', 'REM']

df[good_cols].head(5).style.set_properties(**{'background-color': berkeley_palette['pacific'],'color': berkeley_palette['california_gold'],'border-color': berkeley_palette['berkeley_blue']})\
  .set_caption('Raw Weather Data : First 5 Rows (Only Columns with Data)')

Out[18]:

,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,MA1,REM
0,4013199999,2017-01-02 07:00:00,4,66.058056,-23.135278,2.430000,"ISAFJORDUR, IC",FM-15,99999,V020,"220,1,N,0113,1","00914,1,9,N","009999,1,9,9","+0050,1","+0010,1","99999,9","10180,1,99999,9",MET054METAR BIIS 020700Z 22022G35KT 9999 OVC030 05/01 Q1018=
1,4013199999,2017-01-02 08:13:00,4,66.058056,-23.135278,2.430000,"ISAFJORDUR, IC",FM-15,99999,V020,"220,1,N,0113,1","00914,1,9,N","009999,1,9,9","+0050,1","+0020,1","99999,9","10160,1,99999,9",MET054METAR BIIS 020813Z 22022G36KT 9999 OVC030 05/02 Q1016=
2,4013199999,2017-01-02 09:00:00,4,66.058056,-23.135278,2.430000,"ISAFJORDUR, IC",FM-15,99999,V020,"220,1,N,0103,1","00914,1,9,N","009999,1,9,9","+0050,1","+0020,1","99999,9","10160,1,99999,9",MET054METAR BIIS 020900Z 22020G30KT 9999 OVC030 05/02 Q1016=
3,4013199999,2017-01-02 10:11:00,4,66.058056,-23.135278,2.430000,"ISAFJORDUR, IC",FM-15,99999,V020,"220,1,N,0103,1","00914,1,9,N","009999,1,9,9","+0060,1","+0020,1","99999,9","10140,1,99999,9",MET054METAR BIIS 021011Z 22020G39KT 9999 BKN030 06/02 Q1014=
4,4013199999,2017-01-02 11:00:00,4,66.058056,-23.135278,2.430000,"ISAFJORDUR, IC",FM-15,99999,V020,"220,1,N,0154,1","00914,1,9,N","009999,1,9,9","+0060,1","+0020,1","99999,9","10130,1,99999,9",MET058METAR BIIS 021100Z 22030G44KT 9999 -RA BKN030 06/02 Q1013=


In [0]:
df = ks.sql("with cte as (select distinct station, longitude, latitude from weather_tbl where station is not null) select station, count(*) as total from cte group by station having total > 1 order by 2 desc").to_koalas()

x, y = df.station.to_numpy(),  df.total.to_numpy()
#marker_colors = [berkeley_palette['pacific'] if x > 80.0 else berkeley_palette['lawrence'] for x in missing_subset.missing_pct.to_numpy()]

fig = go.Figure(data=[go.Bar(x=(x+'-'), y=y, hovertext=y)])
fig.update_traces(marker_color=berkeley_palette['pacific'], marker_line_color=berkeley_palette['berkeley_blue'],
                  marker_line_width=1.5, opacity=0.7)
fig.update_layout(title_text='[Weather]: Stations w/ Multiple Lat/Long Locations', title_font_size = 24,
                  height=650, width=1800,
                  yaxis=dict(title='Total (Unique) Lat/Long Locations', titlefont_size=16, tickfont_size=14, color=berkeley_palette['berkeley_blue']),
                  xaxis=dict(title='Station Code', titlefont_size=16, color=berkeley_palette['berkeley_blue'], tickangle=90),
                  plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [0]:
del df

# `STATIONS` Data Analysis

In [0]:
# load raw weather data into koalas dataframe and print raw info about shape and sparsity
df = ks.sql("select * from stations_tbl").to_koalas()
print(df.info())

/databricks/python/lib/python3.7/site-packages/databricks/koalas/generic.py:369: FutureWarning:

`get_dtype_counts` has been deprecated and will be removed in a future version. For DataFrames use `.dtypes.value_counts()

<class 'databricks.koalas.frame.DataFrame'>
Index: 29771 entries, 0 to 29770
Data columns (total 11 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 usaf 29771 non-null object
 1 wban 29771 non-null object
 2 name 28982 non-null object
 3 country 28833 non-null object
 4 state 6742 non-null object
 5 call 10887 non-null object
 6 lat 28567 non-null object
 7 lon 28566 non-null object
 8 elev 28473 non-null object
 9 begin 29771 non-null object
 10 end 29771 non-null object
dtypes: object(11)None

### Raw data profile:
 - 29,771 rows
 - 11 columns
   - `11 x object`

In [0]:
for c in df.columns:
  print(f"{c} - unique values: {len(df[c].unique())}")

usaf - unique values: 26250
wban - unique values: 3105
name - unique values: 26497
country - unique values: 253
state - unique values: 74
call - unique values: 8148
lat - unique values: 11578
lon - unique values: 16905
elev - unique values: 5309
begin - unique values: 7301
end - unique values: 7150

In [0]:
# evaluate missing data
missing = df.isnull().sum(axis=0).reset_index()
missing.columns = ['column', 'total_missing']
missing['missing_pct'] = missing.total_missing / df.shape[0] * 100
missing = missing.sort_values(by=['total_missing','column']).reset_index(drop = True)
missing['category'] = ['No missing data'] * missing.shape[0]
missing.loc[((missing['missing_pct'] > 0.0) & (missing['missing_pct'] <= 40.0)), 'category'] = 'Up to 40% missing data'
missing.loc[((missing['missing_pct'] > 40.0) & (missing['missing_pct'] <= 80.0)), 'category'] = 'Up to 80% missing data'
missing.loc[missing['missing_pct'] > 80.0, 'category'] = 'Over 80% missing data'
#missing.head(5).style.set_properties(**{'background-color': berkeley_palette['california_gold'], 'color': berkeley_palette['berkeley_blue'], 'border-color': 'white'})

cats = missing.groupby('category').column.count().to_frame()
cats.columns = ['Category Count']

cats.style.bar(color = berkeley_palette['founders_rock'], align = 'mid')\
  .set_caption('Distribution of missing data per column in Stations')

Out[41]:

,Category Count
category,
Up to 80% missing data,2
No missing data,4
Up to 40% missing data,5


In [0]:
missing_subset = missing[missing.missing_pct > 0.0].copy().sort_values(by='missing_pct')
x, y, h = missing_subset['column'].to_numpy(),  missing_subset.missing_pct.to_numpy(), missing_subset.total_missing.to_numpy()
marker_colors = [berkeley_palette['pacific'] if x > 60.0 else berkeley_palette['lawrence'] for x in missing_subset.missing_pct.to_numpy()]

fig = go.Figure(data=[go.Bar(x=x, y=y, hovertext=h)])
fig.update_traces(marker_color=marker_colors, marker_line_color=berkeley_palette['berkeley_blue'],
                  marker_line_width=1.5, opacity=0.7)
fig.update_layout(title_text='[Stations]: Columns w/ Missing Values', title_font_size = 24,
                  height=450, width=800,
                  yaxis=dict(title='% Missing Values', titlefont_size=16, tickfont_size=14, color=berkeley_palette['berkeley_blue']),
                  xaxis=dict(title='Column Names', titlefont_size=16, color=berkeley_palette['berkeley_blue']),
                  plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [0]:
fig = px.histogram(df.to_pandas(), x="elev", marginal="violin",
                   hover_data=df.columns, color_discrete_sequence=[berkeley_palette['berkeley_blue']],
                   nbins=100, opacity=0.7, histnorm='probability')

fig.update_layout(title_text='Station Elevation Distribution', height=500, width=1000, plot_bgcolor='rgba(0,0,0,0)',
                 xaxis=dict(title="Elevation"), yaxis=dict(title='Probability'))
fig.show()